In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import *
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv("../../data/sample01/application_record.csv", encoding = 'utf-8')
record = pd.read_csv("../../data/sample01/credit_record.csv", encoding = 'utf-8')

In [ ]:
print(data.shape)
print(record.shape)

In [ ]:
data.sort_values(by=['ID'])

In [ ]:
data.groupby(['ID']).count()

In [ ]:
record.head()

In [ ]:
#agrupar los usuarios y obtener el minimo valor del balance del mes
begin_month=pd.DataFrame(record.groupby(["ID"])["MONTHS_BALANCE"].agg(min))
begin_month

In [ ]:
#renombrar la columna
begin_month=begin_month.rename(columns={'MONTHS_BALANCE':'begin_month'})
begin_month

In [ ]:
#fusionar el dataframe principal con los valores de la columna begin_month
new_data=pd.merge(data,begin_month,how="left",on="ID")

In [ ]:
new_data

In [ ]:
new_data.shape

#### Generalmente, los usuarios en riesgo deberían estar en el 3%, por lo que elijo a los usuarios que tienen más de 60 días de retraso como usuarios de riesgo objetivo. Esas muestras están marcadas como '1', de lo contrario, son '0'.

>ID:
                Número de cliente
>MONTHS_BALANCE:
                 El mes de los datos extraídos es el punto de partida, hacia atrás,
>                * 0 es el mes actual,
>                * -1 es el mes anterior, y así sucesivamente
>ESTADO
>                0: 1-29 días de atraso
>                1: 30-59 días de atraso
>                2: 60-89 días de atraso
>                3: 90-119 días de atraso
>                4: 120-149 días de atraso
>                5: Deudas vencidas o incobrables, cancelaciones por más de 150 días
>                C: Pagado ese mes
>                X: sin préstamo durante el mes

In [ ]:
record['dep_value'] = None
record['dep_value'][record['STATUS'] =='2']='Yes'
record['dep_value'][record['STATUS'] =='3']='Yes'
record['dep_value'][record['STATUS'] =='4']='Yes'
record['dep_value'][record['STATUS'] =='5']='Yes'

In [ ]:
record.head()

In [ ]:
record.tail()

In [ ]:
# contar los valores dep_value agrupados por ID
cpunt=record.groupby('ID').count()
cpunt.head()

In [ ]:
# Yes -> los clientes que tienen mas de 60 dias de retraso
cpunt['dep_value'][cpunt['dep_value'] > 0]='Yes'
cpunt['dep_value'][cpunt['dep_value'] == 0]='No'

In [ ]:
cpunt.head()

In [ ]:
cpunt.tail()

In [ ]:
cpunt = cpunt[['dep_value']]

In [ ]:
cpunt.head()

In [ ]:
cpunt.shape

In [ ]:
# fusionar new_data con dep_value (variable a predecir)
new_data=pd.merge(new_data,cpunt,how='inner',on='ID')

In [ ]:
new_data.head()

In [ ]:
new_data.shape

In [ ]:
new_data['target']=new_data['dep_value']
new_data.loc[new_data['target']=='Yes','target']=1
new_data.loc[new_data['target']=='No','target']=0

In [ ]:
new_data

In [ ]:
print(cpunt['dep_value'].value_counts())
cpunt['dep_value'].value_counts(normalize=True)

#### Features

In [ ]:
#renombrar columnas
new_data.rename(columns={'CODE_GENDER':'Gender','FLAG_OWN_CAR':'Car','FLAG_OWN_REALTY':'Reality',
                         'CNT_CHILDREN':'ChldNo','AMT_INCOME_TOTAL':'inc',
                         'NAME_EDUCATION_TYPE':'edutp','NAME_FAMILY_STATUS':'famtp',
                        'NAME_HOUSING_TYPE':'houtp','FLAG_EMAIL':'email',
                         'NAME_INCOME_TYPE':'inctp','FLAG_WORK_PHONE':'wkphone',
                         'FLAG_PHONE':'phone','CNT_FAM_MEMBERS':'famsize',
                        'OCCUPATION_TYPE':'occyp'
                        },inplace=True)

In [ ]:
new_data.head()

## Identificar y manejar valores

In [ ]:
#crear dataframe con valores booleanos para validar los missing value
missing_data = new_data.isnull()


In [ ]:
# validamos los missing value para todas las columnas excepto el ID
for column in missing_data.columns.values.tolist():
    if column != 'ID':
        print(column)
        print (missing_data[column].value_counts())
        print("")

In [ ]:
# numero de usuarios por ocupacion
new_data.groupby(["occyp"])['ID'].count().sort_values(ascending = False)

In [ ]:
#numero de usuarios por ocupacion# promedio de sueldos anuales por ocupacion (verificar relacion directa)
plot = new_data['occyp'].value_counts().plot(kind='bar',title='Tipos de Ocupacion')

In [ ]:
new_data.groupby(["occyp"])['inc'].mean().sort_values(ascending = False)

opcion 1:
Proceso que recorra todas las filas, al encontrar un ocupacion=NaN, verificar en que rango de sueldo se encuentra para asignarle un ocupacion de acuerdo a sus ingresos anuales
opcion 2:
Eliminar las columnas o filas que contengan NaN

In [ ]:
# resguardamos la data hasta este punto# opcion 2, eliminar valores perdidos
df_bkp=new_data

In [ ]:
# opcion 2, eliminar valores perdidos
new_data.dropna()
new_data = new_data.mask(new_data == 'NULL').dropna()


In [ ]:
# verificamos la dimension del dataset
new_data.shape

In [ ]:
new_data.head()

analizando datos

In [ ]:
# numero de aprobados por genero
g = sns.catplot(x="target",col="Gender",
                data=new_data, kind="count",
                height=4, aspect=0.7);

In [ ]:
# grafico de los promedios de sueldos por ocupacion
a=new_data.groupby(["occyp"])['inc'].mean().sort_values(ascending = False).plot(kind='barh')

In [ ]:
plot = new_data['inctp'].value_counts().plot(kind='bar', title='Usuarios evaluados')
plot.grid()

In [ ]:
plot = (100 * new_data['inctp'].value_counts() / len(new_data['inctp'])).plot(
kind='bar', title='Usuarios evaluados %')
plot.grid()

In [ ]:
# Ingresos anuales por Ocupacion
plt.figure(figsize=(8,10))
sns.barplot(x=new_data['inc'], y= new_data['occyp'])
plt.ylabel('Ocupacion')
plt.xlabel('Ingreso Anual')
plt.show()

#### Relacionando variables categóricas

In [ ]:
# Tabla de contingencia ingreso anuales / ocupacion
pd.crosstab(index=new_data['occyp'],
            columns=new_data['inc'], margins=True)

In [ ]:
# tabla de contingencia en porcentajes relativos total
pd.crosstab(index=new_data['occyp'], columns=new_data['inc'],
            margins=True).apply(lambda r: r/len(new_data) *100,
                                axis=1)

In [ ]:
# Categorical variables
sns.boxplot(x="dep_value", y="inc", data=new_data)

#### Veamos que las distribuciones de dep_value e ingresos anuales entre las diferentes categorías de dep_value no tienen una superposición significativa.


### Formato correcto de los datos

In [ ]:
new_data.dtypes

In [ ]:
#new_data['Gender']=new_data['dep_value']
new_data.loc[new_data['Gender']=='Yes','target']=1
new_data.loc[new_data['Gender']=='No','target']=0

In [ ]:
# tabla de contingencia en porcentajes relativos total
pd.crosstab(index=new_data['dep_value'], columns=new_data['inc'],
            margins=True).apply(lambda r: r/len(new_data) *100,
                                axis=1)